<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/BlockChain/python_blockchain_training_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%writefile hash_util.py
import hashlib as hl
import json

def hash_string_256(string):
  return hl.sha256(string).hexdigest()

def hash_block(block):
  return hash_string_256(json.dumps(block, sort_keys=True).encode())


Overwriting hash_util.py


In [14]:
%%writefile blockchain.py
# try except block : 
#####################################################
import functools
import hashlib as hl
import json
from collections import OrderedDict

from hash_util import hash_string_256, hash_block

#global variable
MINING_REWARD = 10

blockchain = []

open_transactions = []


owner = 'Sumith'
participants = {owner}


def initialise():
  genesis_block = {
          'previous_hash': '', 
          'index':0 , 
          'transactions':[],
          'proof':100
  }
  blockchain = [genesis_block]
  return blockchain

def load_data():
  global blockchain
  global open_transactions 
  try:
    with open('blockchain.txt', mode='r') as f:
      file_content = f.readlines()

      blockchain = json.loads(file_content[0][:-1])
      blockchian = [{ 'previous_hash': block['previous_hash'], 
                    'index' : block['index'], 
                    'transactions' : [OrderedDict([('sender', tx['sender']),
                            ('recipient', tx['recipient']), 
                            ('amount' , tx['amount'])]) for tx in block['transactions']],
                    'proof': block['proof']} for block in blockchain]
    
      open_transactions = json.loads(file_content[1])
      open_transactions = [ OrderedDict([('sender', tx['sender']),
                            ('recipient', tx['recipient']), 
                            ('amount' , tx['amount'])])  for tx in open_transactions]
  except IOError:                    
    print('File not found!')
    blockchain = initialise()
  except ValueError:
    print('Value error!')
  except:
    print('wildcard!')
  finally:
    print('Cleanup!')


load_data()

def save_data():
  try :
    with open('blockchain.txt', mode='w') as f:
      f.write(json.dumps(blockchain))
      f.write('\n')
      f.write(json.dumps(open_transactions))
  except IOError:
    print('Error saving file!')

def get_last_blockchain_value():
  """ returns the last value of the current bloackchain """
  #function to return the value
  if len(blockchain) < 1 :
    return None
  return blockchain[-1]

def verify_transaction(transaction):
  sender_balance = get_balance(transaction['sender'])
  return sender_balance >= transaction['amount']


#adding kwargs to function and setting up default value
#transaction_amount is local varible
def add_transaction(recipient,sender=owner, amount=1.0):
  """ Appends a new value as well as the last blockchain value to teh block

  Arguments:
    : Sender :  sender of coins
    : recipent : recipient of coin
  """
  #appending values to blockchian and retaining the last value
  #transaction = {'sender': sender,
  #               'recipient': recipient, 
  #               'amount' : amount }
  transaction = OrderedDict([('sender', sender),
                            ('recipient', recipient), 
                            ('amount' , amount)])
  if verify_transaction(transaction):
    open_transactions.append(transaction)
    participants.add(sender)
    participants.add(recipient)
    save_data()
    return True
  return False


def valid_proof(transactions, last_hash, proof):
  guess = (str(transactions) + str(last_hash) + str(proof)).encode()
  guess_hash = hash_string_256(guess)
  print(guess_hash)
  return guess_hash[0:2] == '00'

def proof_of_work():
  last_block = blockchain[-1]
  last_hash = hash_block(last_block)
  proof = 0
  while not valid_proof(open_transactions, last_hash, proof):
    proof += 1
  return proof
  

def get_balance(participant):
  tx_sender = [[tx['amount'] for tx in block['transactions'] if tx['sender'] == participant] for block in blockchain]
  open_tx_sender = [tx['amount'] for tx in open_transactions if tx['sender'] == participant]
  tx_sender.append(open_tx_sender)
  amount_sent = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_sender,0)
  tx_recipient = [[tx['amount'] for tx in block['transactions'] if tx['recipient'] == participant] for block in blockchain]
  amount_received = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_recipient,0)
  return  amount_received - amount_sent

def mine_block():
  last_block = blockchain[-1]
  #list comprehensions
  hashed_block = hash_block(last_block)
  proof = proof_of_work() 
  #reward_transaction = {
  #    'sender': 'MINING',
  #    'recipient': owner,
  #    'amount': MINING_REWARD
  #}
  reward_transaction = OrderedDict(
                      [('sender', 'MINING'),
                      ('recipient', owner),
                      ('amount', MINING_REWARD)])
  copied_transactions = open_transactions[:]
  copied_transactions.append(reward_transaction)
  block = {
          'previous_hash': hashed_block, 
          'index' : len(blockchain), 
          'transactions' : copied_transactions,
           'proof': proof
  }
  blockchain.append(block)
  #save_data()
  return True

#using function for reusable code.
def get_transaction_value():
  """ Retruns the input of the user(a new transaction amount) as a float
  """
  tx_recipient = input('Enter the recipent of the transaction: ') 
  tx_amount = float(input('Your trasaction amount please: ')) 
  return (tx_recipient, tx_amount)
  
def get_user_choice():
  user_input = input('Your Choice : ')
  return user_input

def print_blockchain_elements():
  #use for loop to output the blockchain list console
  for block in blockchain:
    print('outputting block : ', block)
  else:
    print('-' * 20)

def verify_chain():
  """verify current blockchain"""
  for (index, block) in enumerate(blockchain):
    if index == 0:
      continue
    if block['previous_hash'] != hash_block(blockchain[index -1]):
      return False
    if not valid_proof(block['transactions'][:-1], block['previous_hash'], block['proof']):
      print('Prood of work invalid')
      return False
  return True

def verify_transactions():
  return all([verify_transaction(tx) for tx in open_transactions])
#  is_valid = True
#  for tx in open_transactions:
#    if verify_transaction(tx):
#      is_valid = True
#    else:
#      is_valid = False
  
waiting_for_input = True

while waiting_for_input:
  print('Please choose')
  print('1: Add a new transaction value')
  print('2: Mine a new block')
  print('3: Output the blockchain blocks')
  print('4: output participants')
  print('5: check transaction validity')
  print('q : Quit ')
  user_choice = get_user_choice()
  if user_choice == '1':
    tx_data = get_transaction_value()
    recipient, amount = tx_data
    if add_transaction(recipient, amount=amount):
      print('Added transaction')
    else:
      print('Tranaction failed')
    print(open_transactions)
  elif user_choice == '2':
    if mine_block():
      open_transactions = []
      save_data()
  elif user_choice == '3':
    print_blockchain_elements()
  elif user_choice == '4':
    print(participants)
  elif user_choice == '5':
    if verify_transactions():
      print("All Transactions are valid")
    else:
      print('There are invalid transaction')
  elif user_choice == 'h':
    if len(blockchain) >=1:
      blockchain[0] = {
          'previous_hash': '', 
          'index':0 , 
          'transactions':[{'sender':'Chris', 'recipient': 'Max', 'amount': 100}]
      }
  elif user_choice == 'q':
    waiting_for_input = False
  else:
    print('Input was invalid, please pick a value from the list!')    
  
  if not verify_chain():
    print_blockchain_elements()
    print('Invalid block chain!')
    break
  print('Balance of {} : {:6.2f}'.format(owner, get_balance(owner)))  
  print('Choice registered !')
else:
  print('User left')

print('Done')


Writing blockchain.py


In [16]:
!python blockchain.py

File not found!
Cleanup!
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: output participants
5: check transaction validity
q : Quit 
Your Choice : 3
outputting block :  {'previous_hash': '', 'index': 0, 'transactions': [], 'proof': 100}
--------------------
Balance of Sumith :   0.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: output participants
5: check transaction validity
q : Quit 
Your Choice : 2
192a620abb0343decb9a29d7d9b911c9450ba4e48cc7fa5256e99155371aedb6
4b5b8289ba0eb454a3a51da1f4f6b2cacb0c41725b8a00031e360a495e8f280a
7d67a07fa8f75a27133c8edd485505db4b374e06a937da73db79aa2e781e4486
4569979386e12581b05dead3abe3d6711ecb933f42c5c635f12e8b33b0dce1bc
e443beb3262f36bb1aaf9afdbacb440941a047b27493e2a30d00a0288db44ab8
3abe3eb5bb76363abd3841a30a44aece6b461c78add963166416e613ea9b3170
ed26a2c7414b523b520cc2743574653521dc5865527ef7d0b3b6dd1e3ccd71cb
ba9b4a0ede4a

In [17]:
!python blockchain.py

Cleanup!
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: output participants
5: check transaction validity
q : Quit 
Your Choice : 3
outputting block :  {'previous_hash': '', 'index': 0, 'transactions': [], 'proof': 100}
outputting block :  {'previous_hash': '68da5267793e1fc7bd98d9cc01cf91e38daa67cbbd7b646bcd99b4dc3df93eb1', 'index': 1, 'transactions': [{'sender': 'MINING', 'recipient': 'Sumith', 'amount': 10}], 'proof': 119}
--------------------
0009d7fd8e3af23b4d72105473696ee9a5956930361a3819d39c7f022064828d
Balance of Sumith :  10.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: output participants
5: check transaction validity
q : Quit 
Your Choice : q
0009d7fd8e3af23b4d72105473696ee9a5956930361a3819d39c7f022064828d
Balance of Sumith :  10.00
Choice registered !
User left
Done


In [0]:
rm blockchain.txt

In [11]:
!cat blockchain.txt

cat: blockchain.txt: No such file or directory
